In [100]:
# 환경 변수 불러오기
from dotenv import load_dotenv
load_dotenv()

True

In [101]:
from langchain_teddynote import logging
logging.langsmith("CLASS")

LangSmith 추적을 시작합니다.
[프로젝트명]
CLASS


In [102]:
# 필요한 모듈 및 클래스 임포트
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context, conditional
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from ragas.testset.extractor import KeyphraseExtractor
from ragas.testset.docstore import InMemoryDocumentStore


In [103]:
import pandas as pd
import ast
from datasets import Dataset

In [146]:
from PyPDF2 import PdfReader, PdfWriter

def add_metadata_to_pdf(input_pdf_path, output_pdf_path, metadata):
    # 기존 PDF 파일 읽기
    reader = PdfReader(input_pdf_path)
    writer = PdfWriter()

    # 모든 페이지를 복사
    for page in reader.pages:
        writer.add_page(page)

    # 메타데이터 추가
    writer.add_metadata(metadata)

    # 새로운 PDF로 저장
    with open(output_pdf_path, "wb") as output_pdf:
        writer.write(output_pdf)
    print(f"메타데이터가 추가된 PDF 저장 완료: {output_pdf_path}")

# 입력 및 출력 파일 경로
input_pdf = "data/converted_data.pdf"          # 기존 PDF 파일 경로
output_pdf = "data/converted_data_with_metadata.pdf"  # 수정된 PDF 파일 저장 경로

# 추가할 메타데이터
metadata = {
    "/Title": "Jump to Phython",          # 제목
    "/Author": "Park",                  # 작성자
    "/Subject": "Python",    # 주제
    "/Keywords": "Python, Book",  # 키워드
    "/Creator": "PyPDF2 Library",            # 생성자
}

# 메타데이터 추가 함수 호출
add_metadata_to_pdf(input_pdf, output_pdf, metadata)


메타데이터가 추가된 PDF 저장 완료: data/converted_data_with_metadata.pdf


In [136]:
# loader 객체 할당
from langchain_community.document_loaders import PDFPlumberLoader
loader = PDFPlumberLoader('data/converted_data_with_metadata.pdf')


In [137]:
# 파일 로드
docs = loader.load()

In [138]:
# 메타데이터에 filename 추가
for doc in docs:
    doc.metadata['filename'] = doc.metadata['source']

In [139]:
# 생성기 할당
generator_llm = ChatOpenAI(model = 'gpt-4o')

# 비평기 할당
critic_llm = ChatOpenAI(model = 'gpt-4o')

# 임베딩 모델 할당
embeddings = OpenAIEmbeddings(model = 'text-embedding-3-small')

In [140]:
# 텍스트 분할기 할당
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

# 텍스트 분할
# split_documents = splitter.split_documents(docs)

In [141]:
# 구문 추출기 생성을 위한 모델 호출(RAGAS와의 호환을 위한 Wrapper)
langchain_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o-mini"))

# 구문추출기 : 문서 핵심 정보 식별 및 추출 역할
keyphrase_extractor = KeyphraseExtractor(llm=langchain_llm)

# 임베딩 모델도 RAGAS와의 호환을 위해 Wrapper 적용
ragas_embeddings = LangchainEmbeddingsWrapper(embeddings)


In [142]:
# 문서를 저장하고, 질문 생성 및 평가에 필요한 데이터를 제공하는 저장소
docstore = InMemoryDocumentStore(
    splitter=splitter, 
    embeddings=ragas_embeddings, 
    extractor=keyphrase_extractor
)

In [143]:
docstore

InMemoryDocumentStore(splitter=<langchain_text_splitters.character.RecursiveCharacterTextSplitter object at 0x0000025232C9CE10>, nodes=[], node_embeddings_list=[], node_map={}, run_config=RunConfig(timeout=180, max_retries=10, max_wait=60, max_workers=16, exception_types=(<class 'Exception'>,), log_tenacity=False, seed=42))

In [144]:
# 생성기 만들기(데이터셋 생성기)
# 생성하면서 동시에 평가 가능
generator = TestsetGenerator.from_langchain(
    generator_llm,      # 생성기
    critic_llm,         # 판별기
    ragas_embeddings,   # 임베딩 모델
    docstore = docstore # 문서 저장소
)

# 질문 유형 분포 결정
distributions = {"simple": 0.4, "reasoning": 0.2, "multi_context": 0.2, "conditional": 0.2}
# simple : 간단하고 직관적 질문 / reasoning : 논리적 추론 필요 질문
# multi_context : 여러 맥락 결합 질문 / conditional : 특정 조건 답변 달라지는 질문


In [145]:
testset = generator.generate_with_langchain_docs(
    documents = docs,
    test_size = 10,
    distributions = distributions,
    with_debugging_logs = True,  
    # 로그 활성화, 출력 과정 내역 확인
    raise_exceptions = True
    # 예외가 발생해도 계속 진행, 예외 발생 시 문서가 처리되지 않거나 문서에 기록됨
)

embedding nodes:   0%|          | 0/738 [00:00<?, ?it/s]

AttributeError: 'str' object has no attribute 'docstore'

In [ ]:
# from langchain.chat_models import ChatOpenAI
from langchain_openai import ChatOpenAI
from ragas.testset.generator import TestsetGenerator

# LLM 초기화
generator_llm = ChatOpenAI(model="gpt-4o", temperature=0)  # 적절한 모델과 설정
critic_llm = ChatOpenAI(model="gpt-4o", temperature=0)

# 메시지 예시
messages = [
    {"role": "system", "content": "당신은 주어진 데이터를 활용해서 질문에 따라 답변을 수행하는 친절한 AI 용어집과 퀴즈를 제공하는 학습 비서입니다."},
    {"role": "user", "content": "당신은 주어진 context에서 주어진 question에 답하는 것을 수행합니다."}
]

# TestsetGenerator 초기화
generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    ragas_embeddings,
    docstore = docstore
)

# 질문 유형 분포 설정
distributions = {"simple": 0.4, "reasoning": 0.2, "multi_context": 0.2, "conditional": 0.2}

# 테스트셋 생성
try:
    testset = generator.generate_with_langchain_docs(
        documents=docs,  # 올바른 형식의 문서 제공
        test_size=10,
        distributions=distributions,
        with_debugging_logs=True,
        raise_exceptions=False
    )
    # 결과 저장
    test_df = testset.to_pandas()
    test_df.to_csv("data/ragas_testset.csv", index=False)
    print("테스트셋 생성 완료!")
except TypeError as e:
    print(f"TypeError 발생: {e}")
except Exception as e:
    print(f"다른 오류 발생: {e}")


TypeError 발생: BaseChatModel.generate() missing 1 required positional argument: 'messages'


In [69]:
# 테스트셋 생성
testset = generator.generate_with_langchain_docs(
    documents=docs, # 테스트 질문을 생성하기 위한 입력 문서
    test_size=10,    # 질문 세트 크기 지정
    distributions=distributions,    # 질문 유형 분포
    with_debugging_logs=True,   # 디버깅 로그 활성화
    # raise_exceptions=True
    raise_exceptions=False    # 오류 발생해도 계속 진행, 실패한 작업은 로그로 기록되거나 스킵
)

AttributeError: 'str' object has no attribute 'docstore'

In [ ]:
# 생성기 만들기
generator = TestsetGenerator.from_langchain(
    generator_llm,      #생성기(질문을 생성)
    critic_llm,         #판별기(생성된 질문의 품질을 검토)
    ragas_embeddings,   #임베딩 모델
    docstore=docstore   #문서저장소
)

# 질문 유형 분포 결정
distributions = {"simple": 0.4, "reasoning": 0.2, "multi_context": 0.2, "conditional": 0.2}
# simple : 간단하고 직관적 질문 / reasoning : 논리적 추론 필요 질문
# multi_context : 여러 맥락 결합 질문 / conditional : 특정 조건 답변 달라지는 질문

# 테스트셋 생성
testset = generator.generate_with_langchain_docs(
    documents=docs, # 테스트 질문을 생성하기 위한 입력 문서
    test_size=10,    # 질문 세트 크기 지정
    distributions=distributions,    # 질문 유형 분포
    with_debugging_logs=True,   # 디버깅 로그 활성화
    # raise_exceptions=True
    raise_exceptions=False    # 오류 발생해도 계속 진행, 실패한 작업은 로그로 기록되거나 스킵
)

# 생성된 테스트셋을 pandas DataFrame형식으로 변환
test_df = testset.to_pandas()

# csv 파일로 저장 (인덱스는 포함하지 않음)
test_df.to_csv("data/ragas_testset.csv", index=False)



TypeError: BaseChatModel.generate() missing 1 required positional argument: 'messages'

In [ ]:
# 테스트셋 로드 및 변환
df = pd.read_csv("data/ragas_testset.csv")
# DataFrame을 Hugging Face Dataset 형태로 변환 (자연어 처리작업에 최적화된 구조)
test_dataset = Dataset.from_pandas(df)

# contexts열 데이터를 문자열 -> 리스트 형식으로 변환 함수 정의
def convert_to_list(example):
    contexts = ast.literal_eval(example['contexts'])
    return {'contexts': contexts}
# 문자열 -> 리스트 함수 적용
test_dataset = test_dataset.map(convert_to_list)


FileNotFoundError: [Errno 2] No such file or directory: 'data/ragas_testset.csv'

In [ ]:
# 데이터 로드
data_path = "data/combined_keyvalue_form.json"
loader = JSONLoader(data_path, json_key="data")

# 파일 로드
docs = loader.load()

# 텍스트 분할기 할당
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

# 텍스트 분할
split_documents = splitter.split_documents(docs)

# 벡터 저장소 생성
vectorstore = FAISS.from_documents(split_documents, embeddings=embeddings)

# 문서 검색기 생성
retriever = vectorstore.as_retriever()

# 프롬프트 템플릿 생성
prompt_template = PromptTemplate.from_template(
    """
    당신은 질문에 따라 답변을 수행하는 친절한 AI 용어집과 퀴즈를 제공하는 학습 비서입니다. 
    당신은 주어진 context에서 주어진 question에 답하는 것을 수행합니다. 검색된 결과인 다음 context를 사용하여 질문인 question에 답하세요.
    만약, context에서 답을 찾을 수 없거나, 답을 모른다면 '주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다.'라고 한글로 답변해 주세요.
    이름이나 기술적인 용어는 번역하지 않고 그대로 출력해주세요.

    Context: {context}
    Question: {question}
    Answer:
    """
)

# LLM 설정
llm = ChatOpenAI(model="gpt-4o", temperature=0)

# Chain 생성
chain = (
    {'context' : retriever, 'question' : RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

TypeError: JSONLoader.__init__() got an unexpected keyword argument 'json_key'

In [ ]:
# 질문 리스트 생성
batch_dataset=[]
for question in test_dataset['question']:
    batch_dataset.append(question)

# 질문 리스트에 대한 체인 한꺼번에 실행
answer = chain.batch(batch_dataset)

# 생성된 답변 열 테스트 데이터셋에 추가
if 'answer' in test_dataset.column_names :
    test_dataset = test_dataset.remove_columns(['answer']).add_column('answer', answer)
else : 
    test_dataset = test_dataset.add_column('answer', answer)


NameError: name 'test_dataset' is not defined

In [ ]:
# RAG 성능 평가
# metrics = [answer_relevancy, faithfulness, context_recall, context_precision]
result = evaluate(
    dataset = test_dataset, # 데이터셋
    metrics = [ # 성능평가
        context_recall,# 모델이 문맥에서 중요한 정보를 잘 회상, 재현했는가
        faithfulness,  # 모델의 답변이 문맥에 기반해서 사실적인가(factual)
        answer_relevancy, # 전체 질문에 모델의 답변이 질문과 얼마나 관련성이 있는가
        context_precision # 모델이 문맥에서 필요한 정보를 정확히 활용했는가
    ]
)
result_df = result.to_pandas()
result_df.loc[:,'context_recall':'context_precision']

# 결과 저장
result_df.to_csv("data/ragas_evaluation.csv", index=False)

print("평가 완료. 결과는 'data/ragas_evaluation.csv'에 저장되었습니다.")
